In [1]:
import pandas as pd

# Read File
data = pd.read_csv('dataset_1.csv')

# View DataFrame
data

,Unnamed: 0,title,ingredients,directions
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish...."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ..."
...,...,...,...,...
223111,223111,Hot Rolls,"[""1 3/4 c. warm water"", ""1/2 c. sugar"", ""1 Tbs...","[""Combine water, sugar, salt and butter, yeast..."
223112,223112,Cheesy Vegetable Casserole,"[""1/2 lb. American cheese"", ""1/2 c. butter"", ""...","[""Cut cheese into cubes and place in saucepan ..."
223113,223113,Volcanic Shake,"[""2 medium bananas"", ""4 ice cubes"", ""1/2 c. yo...","[""Put all ingredients in blender. Blend on low..."
223114,223114,Yogurt Popsicles,"[""2 cartons plain yogurt"", ""1 (12 oz.) can con...","[""Combine ingredients in a bowl. Mix well. Fre..."


In [2]:
import re
from fractions import Fraction

def convert_fractions(text):
    # Match optional whole number + fraction, or just fraction
    pattern = r'(?:(\d+)\s+)?(\d+)/(\d+)'

    def replacer(match):
        whole = int(match.group(1)) if match.group(1) else 0
        numerator = int(match.group(2))
        denominator = int(match.group(3))
        decimal = whole + Fraction(numerator, denominator)
        return "{:.10g}".format(float(decimal))  # Clean, no trailing zeros

    return re.sub(pattern, replacer, text)

In [3]:
import pandas as pd
from fractions import Fraction
import re

# Conversioni fisse
CONVERSIONS = {
    "oz": 30,      # once → grammi
    "lb": 450,      # libbre → grammi
    "pt": 475,      # pinte → millilitri
    "qt": 950,      # quart → millilitri
    "inch": 2.5        # pollici → centimetri
}



In [4]:
import re

def convert_units(text):
    if pd.isna(text):
        return text

    # Convert oz, lb, pt, qt (formato: "4 oz", "2.5 lb", ecc.)
    for unit, factor in CONVERSIONS.items():
        pattern = r'(\d+(\.\d+)?)\s*' + unit
        text = re.sub(pattern, lambda m: f"{round(float(m.group(1)) * factor, 1)} {unit_to_metric(unit)}", text)

    # Converte misure in pollici nei pan
    text = re.sub(r'(\d+)\s*x\s*(\d+)[-\s]*inch', lambda m: f"{int(m.group(1)) * CONVERSIONS['inch']:.0f} x {int(m.group(2)) * CONVERSIONS['inch']:.0f} cm", text)

    return text

def convert_fahrenheit_to_celsius(text):
    # Match degrees like 275°, 275 °F, 275\u00b0F (escaped), etc.
    pattern = r'(\d+)\s*(?:°|\\u00b0)'

    def replacer(match):
        fahrenheit = int(match.group(1))
        celsius = round((fahrenheit - 32) * 5 / 9)
        return f"{celsius}°C"

    return re.sub(pattern, replacer, text)

def unit_to_metric(unit):
    return {
        "oz": "g",
        "lb": "g",
        "pt": "ml",
        "qt": "ml",
        "inch": "cm"
    }[unit]


In [5]:
df = pd.read_csv('dataset_1.csv')

df['ingredients_converted'] = df['ingredients'].apply(convert_fractions)
df['directions_converted'] = df['directions'].apply(convert_fractions)

df['ingredients_converted'] = df['ingredients_converted'].apply(convert_units)
df['directions_converted'] = df['directions_converted'].apply(convert_units)

df['directions_converted'] = df['directions_converted'].apply(convert_fahrenheit_to_celsius)

df

,Unnamed: 0,title,ingredients,directions,ingredients_converted,directions_converted
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""1 c. firmly packed brown sugar"", ""0.5 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish...."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""2 (480.0 g.) pkg. frozen corn"", ""1 (240.0 g....","[""In a slow cooker, combine all ingredients. C..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""1 large whole chicken"", ""2 (315.0 g.) cans c...","[""Boil and debone chicken."", ""Put bite size pi..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""1 c. peanut butter"", ""0.75 c. graham cracker...","[""Combine first four ingredients and press in ..."
...,...,...,...,...,...,...
223111,223111,Hot Rolls,"[""1 3/4 c. warm water"", ""1/2 c. sugar"", ""1 Tbs...","[""Combine water, sugar, salt and butter, yeast...","[""1.75 c. warm water"", ""0.5 c. sugar"", ""1 Tbsp...","[""Combine water, sugar, salt and butter, yeast..."
223112,223112,Cheesy Vegetable Casserole,"[""1/2 lb. American cheese"", ""1/2 c. butter"", ""...","[""Cut cheese into cubes and place in saucepan ...","[""225.0 g. American cheese"", ""0.5 c. butter"", ...","[""Cut cheese into cubes and place in saucepan ..."
223113,223113,Volcanic Shake,"[""2 medium bananas"", ""4 ice cubes"", ""1/2 c. yo...","[""Put all ingredients in blender. Blend on low...","[""2 medium bananas"", ""4 ice cubes"", ""0.5 c. yo...","[""Put all ingredients in blender. Blend on low..."
223114,223114,Yogurt Popsicles,"[""2 cartons plain yogurt"", ""1 (12 oz.) can con...","[""Combine ingredients in a bowl. Mix well. Fre...","[""2 cartons plain yogurt"", ""1 (360.0 g.) can c...","[""Combine ingredients in a bowl. Mix well. Fre..."
